## 測試模型

In [1]:
from unet import UNet
from dataset import getAllDataPath, CTMRI_ImageDataset
from criterion import DiceLoss, FocalLoss
import numpy as np
import albumentations as A
import torch
from torch.nn import functional as F

CT_data train (2299, 2)
CT_data test (575, 2)
MRI_data train (123, 2)
MRI_data test (31, 2)
MRI_imgOnly_data train (309,)


In [2]:
save_root = 'saved/0420-1/'
dice_criterion = DiceLoss()
WIDTH = 256
HEIGHT = 256 
BATCH_SIZE = 8
device = 'cuda:0'

filepath = f'{save_root}model_MRI.bin'
print(filepath)
# model_MRI =  Unet(out_sz=(HEIGHT, WIDTH), out_channels=out_channels, activation=nn.Sigmoid(), multi_level=0).to(device)
# model_MRI =  UNet(out_sz=(HEIGHT, WIDTH), num_class=1, activation=None).to(device)
model_MRI =  UNet(out_sz=(HEIGHT, WIDTH), num_class=1, enc_chs=(1,64,128,256,512,1024),
                  activation=None, # nn.Softmax(dim=1)
                 ).to(device) 

model_MRI.load_state_dict(torch.load(filepath)) 


saved/0420-1/model_MRI.bin


<All keys matched successfully>

### load model

In [3]:
root = '/home/jovyan/DA/DATA/TA_data/CHAOS_AIAdatasets/2_Domain_Adaptation_dataset/MRI_testset/'
MRI_testset = getAllDataPath(root, test_split_size=None)


for data in ['MRI_testset']:
    i = eval(data)
    for k in i.keys():
        print(data,k, np.shape(i[k]))


MRI_testset train (160, 2)


In [4]:
target_transform = A.Compose([
#     A.Normalize(p=1, mean=(0.485), std=(0.229)),                         
    A.ToFloat(always_apply=True),
    A.Resize(WIDTH, HEIGHT),
])

dataset_test = CTMRI_ImageDataset(MRI_testset['train'], transform=target_transform) # **如果要正式使用要記得把這裡換成X_test
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
print('BATCH_SIZE:', BATCH_SIZE)
for i in ['dataloader_test']:
    loader = eval(i)
    print('---')
    print(i, loader.__len__())
    print([i.shape for i in iter(loader).next()])
    

BATCH_SIZE: 8
---
dataloader_test 20
[torch.Size([8, 1, 256, 256]), torch.Size([8, 1, 256, 256])]


In [6]:
test_dataloader = dataloader_test

testing_loss = 0.0
# 建立3個block

with torch.no_grad(): # 避免torch計算gradient產生記憶體負擔
    for i, data in enumerate(test_dataloader, 1): 
        torch.cuda.empty_cache()
        image, mask = data

        image = image.to(device)
        mask = mask.to(device)
        outputs = model_MRI(image)
#         outputs = F.sigmoid(outputs)
    
#         samp = outputs[outputs>0.01].detach().cpu().numpy()
#         samp = np.unique(np.around(samp, 2), return_counts=True)
#         print(outputs[outputs>0.05].unique())
#         outputs = F.softmax(outputs, dim=1)
#         for out, ma in zip(outputs, mask):
#             out = out[1]
# #             print(out.shape, ma.shape)
#             out = out.unsqueeze(0)
# #             print(out[ma==1])
#         outputs = outputs[:,1,:,:]
#         outputs = outputs.unsqueeze(1)

        'asd'
        threshold = 0
        outputs[outputs>threshold] = 1.
        outputs[outputs!=1.] = 0.
        outputs = outputs
#         outputs = torch.ones(outputs.data.size()).to(device)
        
#         for o in outputs:
#             print(o.sum())
#         outputs = outputs.argmax(dim=1)

        loss = DiceLoss()(outputs, mask)
        testing_loss += loss
        
loss =  testing_loss/len(test_dataloader)
loss


tensor(0.3254, device='cuda:0')

In [7]:
# 全猜0: diceloss: tensor(0.6115, device='cuda:0')
# 全猜1: diceloss: tensor(0.9462, device='cuda:0')

In [8]:
def mask2rle(img):
    """
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formatted
    """
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [9]:
import pandas as pd
submission = []
for file_list, dataloader in zip([MRI_testset['train']], [dataloader_test]):
#     print(file_list)
    test_list = [['-'.join([str(i[0].split('/')[idx]) for idx in [-4,-3,-1]])] for i in file_list]
#     len(CT_test_list)

    dataloader = iter(dataloader)
    print(len(file_list))
    i = 0
    while 1:
        try:
            image, mask = dataloader.next()
            image = image.to(device)
#             mask = mask.to(device)
            
            outputs= model_MRI(image)
        
            if outputs.data.size()[1] > 1:
                outputs = outputs.transpose(1,-1)
                pred = clf.predict(outputs.reshape(-1,2).detach().cpu().numpy())
                pred = torch.Tensor(pred).to(device)
                outputs = pred.reshape(mask.data.size())#.detach().cpu().numpy()
            else:
                outputs[outputs>threshold] = 1.
                outputs[outputs!=1] = 0.
                outputs = outputs.detach().cpu().numpy()
#                 outputs = outputs.detach().cpu().numpy()
            
            #outputs = outputs#.detach().cpu().numpy()
#             outputs = torch.zeros(image.data.size())
            for out in outputs:
                test_list[i].append(mask2rle(out))

                i += 1
        except StopIteration:
            print(i)
            print('complete')
            break
    submission+=test_list
    assert i==len(test_list)
    
pd.DataFrame(submission, columns=['filename', 'rle']).to_csv('submission.csv', index=False)

160
160
complete


In [10]:
import os
try:
    if get_ipython().__class__.__name__=='ZMQInteractiveShell':
        os.system('jupyter nbconvert testing_submission.ipynb --to python')
except NameError:
    pass

[NbConvertApp] Converting notebook testing_submission.ipynb to python
[NbConvertApp] Writing 5308 bytes to testing_submission.py
